## Exercise 62 - Distinct + Count

Reading : TimeStamp,StockID,Price

**Every 30 seconds** print on the standard output the StockID and the price variation (%) in the **last 30 seconds** of the stock with a price variation greater than 0.5% in the last 30 seconds. 

In [1]:
from pyspark.streaming import StreamingContext

In [ ]:
# create the spark object that wait for the 
# information every two seconds
ssc = StreamingContext(sc,30)

# the stream will be connected to localhost
# through port 9999
lineDStream = ssc.SocketTextStream("localhost",9999)

In [ ]:
# extract all the stock prices
# emit (stockId, (price,price))
priceDStreams = lineDStream.map(lambda line : (line.split(",")[1], (float(line.split(",")[2],float(line.split(",")[2]))

# the vairation is given by (max(price) - min(price)) / min(price)
# so for each stock (key), reduce all the prices
minMaxPricesDStreams  = priceDStreams.reduceByKey(lambda r1, r2: (min(r1[0],r2[0]),max(r1[1],r2[1])))

# compute the variation as showed before
# select the ones with a variation > 0.5%
filteredStocks = minMaxPricesDStreams.mapValues(lambda pair : 100.0* ((pair[1] - pair[0]) / pair[0]))\
                                     .filter(lambda variation : variation > 0.5)
                                               
# print on the standard output
filteredStocks.pprint()

In [ ]:
# start reading the incoming data
ssc.start()

In [ ]:
# stay alive for at most 90 seconds
ssc.awaitTerminationOrTimeout(90)
# stop only the StreamingContext
# but not the SparkContext
ssc.stop(stopSparkContext=False)